In [ ]:
import ee
import geemap

In [ ]:
ee.Authenticate()

ee.Initialize(project='cloud-computing-sean')

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2019-01-01', '2020-12-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .map(mask_s2_clouds)
    .select(['B4', 'B3', 'B2', 'B8'])
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.core.Map()
m.set_center(103.60620151671053,1.4702747100792306, 10)
m.add_layer(dataset.mean(), visualization, 'RGB')
m

Map(center=[1.4702747100792306, 103.60620151671053], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [ ]:


color_nir = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').select(
    ['B4', 'B3', 'B2', 'B8']
)

# Create a geometry representing an export region.
geometry = ee.Geometry.Rectangle([103.60570070513373, 1.1586987006352678, 104.08848306516305, 1.4707748320847587])

In [ ]:
geemap.ee_export_image_to_drive(
    dataset.mean(), description="singapore_gee_nir", folder="singapore_gee", scale=15, region=geometry
)